In [2]:
import random
import sympy
import csv


In [3]:
# Get a random triple (p, a, b) where p is prime and a,b are numbers betweeen 2 and p-1
def get_random_hash_function():
    n = random.getrandbits(64)
    if n < 0: 
        n = -n 
    if n % 2 == 0:
        n = n + 1
    while not sympy.isprime(n):
        n = n + 1
    a = random.randint(2, n-1)
    b = random.randint(2, n-1)
    return (n, a, b)
# hash function for a number
def hashfun(hfun_rep, num):
    (p, a, b) = hfun_rep
    return (a * num + b) % p
# hash function for a string.
def hash_string(hfun_rep, hstr):
    n = hash(hstr)
    return hashfun(hfun_rep, n)

In [4]:
# Class for implementing a count min sketch "single bank" of counters
class CountMinSketch:
    # Initialize with `num_counters`
    def __init__ (self, num_counters):
        self.m = num_counters
        self.hash_fun_rep = get_random_hash_function()
        self.counters = [0]*self.m
    
    # function: increment 
    # given a word, increment its count in the countmin sketch
    def increment(self, word):
        #
        hWord = hash_string(self.hash_fun_rep, word)%self.m
        self.counters[hWord] = self.counters[hWord] + 1
        #return None
        
    # function: approximateCount
    # Given a word, get its approximate count
    def approximateCount(self, word):   
        return self.counters[hash_string(self.hash_fun_rep, word)%self.m]

In [5]:
# We will now implement the algorithm for a bank of k counters
# Initialize k different counters
def initialize_k_counters(k, m): 
    return [CountMinSketch(m) for i in range(k)]
# Function increment_counters
# increment each of the individual counters with the word
def increment_counters(count_min_sketches, word):
    for i in range(len(count_min_sketches)):
        count_min_sketches[i].increment(word)
    # your code here
    
        
# Function: approximate_count
# Get the approximate count by querying each counter bank and taking the minimum
def approximate_count(count_min_sketches, word):
    return min([cms.approximateCount(word) for cms in count_min_sketches])

In [6]:
#get data stream
rows=[]
with open("./archive/groceries - groceries.csv", 'r') as f:
    csvreader = csv.reader(f)
    fields = next(csvreader)
 
    # extracting each data row one by one
    for row in csvreader:
        rows.append(row)
 
    # get total number of rows
    print("Total number of rows: %d"%(csvreader.line_num))


# printing first 5 rows
print('\nFirst 5 rows are:\n')
for row in rows[:5]:
    # parsing each column of a row
    for col in row:
        print("%10s"%col,end=" "),
    print('\n')

Total number of rows: 9836

First 5 rows are:

         4 citrus fruit semi-finished bread  margarine ready soups                                                                                                                                                                                                                                                                                                                     

         3 tropical fruit     yogurt     coffee                                                                                                                                                                                                                                                                                                                                

         1 whole milk                                                                                                                                                                                          

In [7]:
#init counter
cms = initialize_k_counters(20, 55)
uniqueItems = {}
for row in rows:
    for item in row[1:]:
        if item != "":
            uniqueItems.update({item : 0})
            increment_counters(cms, item)

In [8]:
print("Total number of distincts elements",len(uniqueItems))

Total number of distincts elements 169


In [9]:
#get approximate count of each item
for item in uniqueItems:
    uniqueItems.update({item:approximate_count(cms, item)})
sorted_uniqueItems = sorted(uniqueItems.items(), key=lambda x:x[1], reverse=True)

In [10]:
#get top 10 most frequent items
for item in sorted_uniqueItems[:10]:
    print(item, '\n')

('whole milk', 2513) 

('other vegetables', 1903) 

('rolls/buns', 1809) 

('soda', 1715) 

('yogurt', 1372) 

('bottled water', 1117) 

('tropical fruit', 1073) 

('root vegetables', 1072) 

('shopping bags', 1002) 

('pastry', 932) 



In [11]:
singleItems = {}
scms = initialize_k_counters(20, 55)
for row in rows:
 for item in row[1:]:
            if item != "" and row[:1] == ['1']:
                singleItems.update({item : 0})
                increment_counters(scms, item)

In [12]:
for item in singleItems:
    singleItems.update({item:approximate_count(scms, item)})
sorted_singleItems = sorted(singleItems.items(), key=lambda x:x[1], reverse=True)

In [13]:
for item in sorted_singleItems[:5]:
    print(item, '\n')

('canned beer', 260) 

('soda', 156) 

('whole milk', 121) 

('bottled beer', 121) 

('rolls/buns', 109) 

